In [1]:
import requests
from bs4 import BeautifulSoup
import time
from selenium.webdriver import Chrome
import re
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import datetime as dt

delay = 3
browser = Chrome("./chromedriver")
browser.implicitly_wait(delay)
browser.get("https://www.youtube.com/channel/UCN8CPzwkYiDVLZlgD4JQgJQ/videos") # 박막례할머니 동영상 페이지로 들어감
browser.maximize_window()
body = browser.find_element_by_tag_name('body') # body태그 가져오기

num_of_pagedowns = 22
while num_of_pagedowns:
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(2)
    num_of_pagedowns -= 1 #video페이지의 스크롤을 충분히 밑으로 내려놓고

In [2]:
html0 = browser.page_source
html = BeautifulSoup(html0, 'html.parser')
video_list2 = html.select('div #dismissable')

youtube_url = 'http://www.youtube.com'

park_url = []

#썸네일, 유형, 동영상 게시일, 영상길이, 제목, 조회수, 좋아요, 싫어요 를 저장하기 위한 리스트
thumbnail_list = []
category_list = []
upload_date_list = []
video_length_list = []
title_list = []
view_count_list = []
likes_count_list = []
dislikes_count_list = []


print("video_list2의 길이:",len(video_list2))
for i in range(len(video_list2)):
    url = youtube_url + video_list2[i].find('a',{'id': 'thumbnail'})['href']
    park_url.append(url) # 모든 영상의 url을 youtube_url에 붙여서 완전한 url로 만들고 park_url이라는 리스트에 넣어줌

video_list2의 길이: 272


In [3]:
for video in video_list2:
    video_length = video.select_one("ytd-thumbnail-overlay-time-status-renderer.ytd-thumbnail > span.style-scope")
    video_length = video_length.text.replace("\n","").strip()
    video_length_list.append(video_length)
    print(video_length)

8:16
2:01
9:08
7:38
5:01
3:05
6:56
5:54
7:39
3:02
11:00
6:20
9:48
9:46
0:38
5:58
6:21
3:22
6:49
4:19
6:26
9:53
3:00
7:53
7:28
5:31
9:35
3:29
6:25
5:00
8:25
5:00
1:07
7:04
8:12
2:38
4:23
5:36
4:19
8:18
7:15
8:23
6:32
7:21
6:07
5:45
3:41
2:12
2:25
4:57
6:08
5:04
5:21
7:32
5:22
7:07
7:33
5:00
4:45
6:12
4:36
11:21
4:29
1:33
6:29
4:55
9:10
6:45
4:37
6:14
8:52
6:38
5:06
13:57
3:16
6:36
9:20
5:35
2:00
7:53
3:02
6:07
7:27
7:07
6:23
3:47
7:11
6:19
6:07
7:03
6:57
12:30
4:56
6:42
3:40
7:12
3:24
6:32
1:32
7:06
7:59
5:50
6:06
5:39
9:04
5:30
2:55
8:21
8:15
5:50
7:00
6:45
7:57
8:26
5:12
3:54
6:05
2:08
2:42
6:15
6:13
6:35
8:01
5:18
5:46
4:33
4:16
5:41
7:05
6:20
6:50
3:31
5:12
7:59
6:25
8:49
6:32
0:39
4:32
6:03
5:02
7:12
6:57
11:40
6:34
7:13
1:08
8:03
7:21
4:41
6:25
1:27
9:04
8:18
6:13
6:51
7:57
8:06
3:55
8:09
7:49
7:38
6:51
5:32
0:14
6:38
8:50
14:13
6:29
8:22
6:44
4:52
5:49
5:51
5:36
9:19
10:19
4:37
7:20
5:05
7:59
7:35
0:29
6:01
5:42
6:59
6:33
6:23
7:40
7:41
8:54
4:59
11:53
5:50
6:31
4:35
8:08
3:09
10

In [4]:
# 현재로서는 썸네일, 유형, 동영상 게시일, 영상길이, 제목, 조회수, 좋아요, 싫어요, 만 크롤링 하면 될듯!

# park = pd.DataFrame({'thumbnail': [],
#                      'title' : [],
#                      'category': [],
#                      'view_count': [],
#                      'likes_count': [],
#                      'dislikes_count':[],
#                      'video_length':[],
#                      'upload_date': [],
#                      }) #총 8개의 정보

In [5]:
for i in range(0, 20):
    #여기까지는 video페이지에서 수집할 수 있는 썸네일, 제목을 가져온다.
    thumbnail = video_list2[i].find('a',{'id':'thumbnail'}).find('img')['src'] # 그 하위의 img라는 태그를 또 찾고 거기서 src가져와라 # 1
    title = video_list2[i].find('a',{'id':'video-title'}).text # 2 
    
    # 여기서부터 영상 클릭해서 들어간 페이지
    click_url = park_url[i]
    browser.get(click_url)
    time.sleep(1.5) # 페이지를 넘어가야하므로 로딩을 기다려야함
    # body = browser.find_element_by_tag_name('body')
    browser.find_element_by_xpath("//*[@id='more']/yt-formatted-string").click() # 카테고리를 크롤링 하기 위해서 더보기를 눌러주자
    time.sleep(3)
    html0 = browser.page_source
    html = BeautifulSoup(html0, 'html.parser')
   
    # 여기서는 카테고리, 조회수, 좋아요, 싫어요, 영상길이, 동영상 게시일을 크롤링한다 
    category = html.find('div', {'id':'collapsible'}).find('a',{'class':'yt-simple-endpoint'}).text
    view_count = html.find('span', {'class': 'view-count'}).text # 4
    
    # 좋아요와 싫어요는 완전히 똑같은 태그에 들어있다.
    # <파이썬 정규 표현식을 지원하는 >re모듈을 사용해보자
    likes_count = html.find('yt-formatted-string', {'id':'text', 'class':'style-scope ytd-toggle-button-renderer style-text', # 5
    'aria-label':re.compile('좋아요')}).text+'개'
    dislikes_count = html.find('yt-formatted-string', {'id':'text', 'class':'style-scope ytd-toggle-button-renderer style-text', # 6
    'aria-label':re.compile('싫어요')}).text+'개'
    upload_date = html.find('div',{'id':'date'}).find('yt-formatted-string').text # 8
    
    print(i,".","썸네일", thumbnail, "제목:", title, "카테고리",category,"조회수",view_count,"좋아요수",likes_count, "싫어요수",dislikes_count, "게시일", upload_date)
    
    thumbnail_list.append(thumbnail)
    title_list.append(title)
    category_list.append(category)
    view_count_list.append(view_count)
    likes_count_list.append(likes_count)
    dislikes_count_list.append(dislikes_count)
    upload_date_list.append(upload_date)

0 . 썸네일 https://i.ytimg.com/vi/1LW33ylk8wM/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLAKMobJzYv2PTYQsoZ6vJKWonw7lg 제목: 박막례 간장게장 만들기 [박막례 할머니] 카테고리 코미디 조회수 조회수 612,379회 좋아요수 1.9만개 싫어요수 223개 영상길이 3:30 게시일 2020. 1. 5.
1 . 썸네일 https://i.ytimg.com/vi/BnmL_KbnStE/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLAtKUvlAfw_Tr8TbtOVezuPGN3qyg 제목: 핫팩 사세요~ 호~ HOT핫HOT핫팩이 왔어요~❤️ [박막례 할머니] 카테고리 코미디 조회수 조회수 126,393회 좋아요수 7.4천개 싫어요수 31개 영상길이 3:30 게시일 2020. 1. 3.
2 . 썸네일 https://i.ytimg.com/vi/nbQQauKcHmk/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLDVsZI6akM_iSkowKHTchRmfaME5w 제목: 그 연예대상 시상식 브이로그!!!  [박막례 할머니] 카테고리 코미디 조회수 조회수 538,391회 좋아요수 2.4만개 싫어요수 141개 영상길이 3:30 게시일 2019. 12. 29.
3 . 썸네일 https://i.ytimg.com/vi/NrTC1wqiGVQ/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLAyCTp96M5hXLjXPM_UbJUCUw2xpQ 제목: 크리스마스 홈파티 인 파리 [박막례 할머니] 카테고리 코미디 조회수 조회수 246,647회 좋아요수 1만개 싫어요수 73개 영상길이 3:30 게시일 2019. 12. 24.

In [7]:
import pandas as pd
insert_data = pd.DataFrame({
                    'thumbnail': thumbnail_list,
                    'title' : title_list,
                    'category': category_list,
                    'view_count': view_count_list,
                    'likes_count': likes_count_list,
                    'dislikes_count':dislikes_count_list,
                    'upload_date': upload_date_list,
                    }) 
insert_data

,thumbnail,title,category,view_count,likes_count,dislikes_count,upload_date
0,https://i.ytimg.com/vi/1LW33ylk8wM/hqdefault.j...,박막례 간장게장 만들기 [박막례 할머니],코미디,"조회수 612,379회",1.9만개,223개,2020. 1. 5.
1,https://i.ytimg.com/vi/BnmL_KbnStE/hqdefault.j...,핫팩 사세요~ 호~ HOT핫HOT핫팩이 왔어요~❤️ [박막례 할머니],코미디,"조회수 126,393회",7.4천개,31개,2020. 1. 3.
2,https://i.ytimg.com/vi/nbQQauKcHmk/hqdefault.j...,그 연예대상 시상식 브이로그!!! [박막례 할머니],코미디,"조회수 538,391회",2.4만개,141개,2019. 12. 29.
3,https://i.ytimg.com/vi/NrTC1wqiGVQ/hqdefault.j...,크리스마스 홈파티 인 파리 [박막례 할머니],코미디,"조회수 246,647회",1만개,73개,2019. 12. 24.
4,https://i.ytimg.com/vi/8UjKbZQHCBE/hqdefault.j...,"(Eng)장성규, 장동윤 보고 온 썰ㅋㅋ (수영고모 화남) [박막례 할머니]",코미디,"조회수 239,273회",8.7천개,55개,2019. 12. 20.
5,https://i.ytimg.com/vi/W_p_IcrgykM/hqdefault.j...,(Eng)앙리와 우정테스트 [박막례 할머니],코미디,"조회수 286,439회",9.4천개,78개,2019. 12. 18.
6,https://i.ytimg.com/vi/E84VukdjwSk/hqdefault.j...,(Eng)손녀에게 병주고 약주기 (크리스마스 선물) [박막례 할머니],코미디,"조회수 367,790회",1.3만개,151개,2019. 12. 14.
7,https://i.ytimg.com/vi/tusq5S3j190/hqdefault.j...,(Eng) 수영고모 쌈밥집에 장고래 왔다간 썰ㅋㅋㅋㅋ [박막례 할머니],코미디,"조회수 416,715회",1.5만개,84개,2019. 12. 10.
8,https://i.ytimg.com/vi/RvvQSSYh7_s/hqdefault.j...,(Eng)와플기계로 감자전 만들기 [박막례 할머니],코미디,"조회수 832,019회",2.3만개,217개,2019. 12. 5.
9,https://i.ytimg.com/vi/xOwzLidnG8Q/hqdefault.j...,(Eng)박막례 심즈처럼 살기 [박막례 할머니],코미디,"조회수 363,514회",1.7만개,75개,2019. 12. 1.
